<a href="https://colab.research.google.com/github/uvictoli/Machine_Learning/blob/main/Lv4/Lv4_%ED%8A%9C%EB%8B%9D_5_7_%EB%AA%A8%EB%8D%B8_%ED%8A%9C%EB%8B%9D_%EB%B0%8F_Voting_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [↩️ 리스트로 돌아가기](https://dacon.io/competitions/open/235698/overview/description)

# 셀을 실행할 때 팝업이 뜨면 `무시하고 계속하기` 를 눌러주세요


## 데이터 다운로드
---
아래 셀을 실행시켜 데이터를 colab 에 불러옵니다.
셀 실행은 Ctrl + Enter 를 이용해 실행시킬 수 있습니다.

In [1]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-02-19 12:13:08--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2022-02-19 12:13:08--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 172.217.1.206, 2607:f8b0:4004:801::200e
Connecting to drive.google.com (drive.google.com)|172.217.1.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/emngscnv43pm8kil8jv7fo07o3c95vl4/1645272750000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download [following]
--2022-02-19 12:13:09--  https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffks

In [2]:
# 라이브러리 및 데이터 불러오기

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# 데이터를 불러와 학습시킬 준비하기

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Scailing
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])
test['Scaled fixed acidity'] = scaler.transform(test[['fixed acidity']])

# Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])
onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])

onehot = encoder.transform(test[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
test = pd.concat([test, onehot], axis = 1)
test = test.drop(columns = ['type'])

test.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


index  fixed acidity  ...  x0_red  x0_white
0      0            9.0  ...     0.0       1.0
1      1           13.3  ...     1.0       0.0
2      2            6.5  ...     0.0       1.0
3      3            7.2  ...     0.0       1.0
4      4            6.8  ...     0.0       1.0

[5 rows x 15 columns]

In [3]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=9e3b8418d3b16adbede858355708c1b5a5fde5a8294a22afe9ba260a09ea4000
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [4]:
# Bayesian Optimization 불러오기
from bayes_opt import BayesianOptimization

## 👋 실습
---
실습 순서는 다음과 같습니다.
1. Random forest 튜닝
2. XGBoost 튜닝
3. Light GBM 튜닝
4. Voting Claasifier 생성
5. Voting Claasifier 학습 및 예측
---
이번시간에는 실습 량이 많아 Voting Classifier 생성까지만 진행하고 다음 시간에 이어서 진행 할 예정입니다.






### Random forest 튜닝


In [6]:
# X에 학습할 데이터를, y에 목표 변수를 저장해주세요

X = train.drop(columns = ['index', 'quality'])

y = train['quality']



# 랜덤포레스트의 하이퍼 파라미터의 범위를 dictionary 형태로 지정해주세요

## Key는 랜덤포레스트의 hyperparameter이름이고, value는 탐색할 범위 입니다.

rf_parameter_bounds = {

                      'max_depth' : (1,3), # 나무의 깊이

                      'n_estimators' : (30,100),

                      }



# 함수를 만들어주겠습니다.

# 함수의 구성은 다음과 같습니다.

# 1. 함수에 들어가는 인자 = 위에서 만든 함수의 key값들

# 2. 함수 속 인자를 통해 받아와 새롭게 하이퍼파라미터 딕셔너리 생성

# 3. 그 딕셔너리를 바탕으로 모델 생성

# 4. train_test_split을 통해 데이터 train-valid 나누기

# 5 .모델 학습

# 6. 모델 성능 측정

# 7. 모델의 점수 반환



def rf_bo(max_depth, n_estimators):
  rf_params = {
      'max_depth' : int(round(max_depth)),
      'n_estimators' : int(round(n_estimators))
  }

  rf = RandomForestClassifier(**rf_params)

  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)
  
  rf.fit(X_train, y_train)
  
  score = accuracy_score(y_valid, rf.predict(X_valid))
  return score

# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_rf"라는 변수에 Bayesian Optmization을 저장해보세요

BO_rf = BayesianOptimization(
    f = rf_bo, pbounds = rf_parameter_bounds, random_state = 0
)



# Bayesian Optimization을 실행해보세요

BO_rf.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.49     |  2.098    |  80.06    |
|  2        |  0.5391   |  2.206    |  68.14    |
|  3        |  0.5327   |  1.847    |  75.21    |
|  4        |  0.5291   |  1.875    |  92.42    |
|  5        |  0.52     |  2.927    |  56.84    |
|  6        |  0.4727   |  1.018    |  48.01    |
|  7        |  0.4682   |  1.217    |  66.52    |
|  8        |  0.5236   |  2.251    |  68.09    |
|  9        |  0.5491   |  2.166    |  68.29    |
|  10       |  0.4945   |  2.089    |  87.6     |


### XGBoost 튜닝

In [7]:
# X에 학습할 데이터를, y에 목표 변수를 저장해주세요

X = train.drop(columns = ['index', 'quality'])

y = train['quality']



# XGBoost의 하이퍼 파라미터의 범위를 dictionary 형태로 지정해주세요

## Key는 XGBoost hyperparameter이름이고, value는 탐색할 범위 입니다.

xgb_parameter_bounds = {

                      'gamma' : (0,10),

                      'max_depth' : (1,3), # 나무의 깊이

                      'subsample' : (0.5,1)

                      }



# 함수를 만들어주겠습니다.
# 함수의 구성은 다음과 같습니다.
# 1. 함수에 들어가는 인자 = 위에서 만든 함수의 key값들
# 2. 함수 속 인자를 통해 받아와 새롭게 하이퍼파라미터 딕셔너리 생성
# 3. 그 딕셔너리를 바탕으로 모델 생성
# 4. train_test_split을 통해 데이터 train-valid 나누기
# 5 .모델 학습
# 6. 모델 성능 측정
# 7. 모델의 점수 반환



def xgb_bo(gamma,max_depth, subsample):
  xgb_params = {
      'gamma' : int(round(gamma)),
      'max_depth' : int(round(max_depth)),
      'subsample' : int(round(subsample))
  }

  xgb = XGBClassifier(**xgb_params)

  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)

  xgb.fit(X_train, y_train)

  score = accuracy_score(y_valid, xgb.predict(X_valid))
  return score


# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_xgb"라는 변수에 Bayesian Optmization을 저장해보세요

BO_xgb = BayesianOptimization(
    f = xgb_bo, pbounds = xgb_parameter_bounds, random_state = 0
    )



# Bayesian Optimization을 실행해보세요

BO_xgb.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   |   gamma   | max_depth | subsample |
-------------------------------------------------------------
|  1        |  0.5373   |  5.488    |  2.43     |  0.8014   |
|  2        |  0.5282   |  5.449    |  1.847    |  0.8229   |
|  3        |  0.5436   |  4.376    |  2.784    |  0.9818   |
|  4        |  0.5609   |  3.834    |  2.583    |  0.7644   |
|  5        |  0.5664   |  5.68     |  2.851    |  0.5355   |
|  6        |  0.5382   |  6.692    |  2.421    |  0.9232   |
|  7        |  0.5782   |  2.303    |  2.915    |  0.6916   |
|  8        |  0.5773   |  2.34     |  2.909    |  0.7521   |
|  9        |  0.002727 |  2.481    |  2.122    |  0.5      |
|  10       |  0.6082   |  1.558    |  3.0      |  1.0      |


### LGBM 튜닝


In [10]:
# X에 학습할 데이터를, y에 목표 변수를 저장해주세요

X = train.drop(columns = ['index', 'quality'])

y = train['quality']



# LGBM의 하이퍼 파라미터의 범위를 dictionary 형태로 지정해주세요

## Key는 LGBM hyperparameter이름이고, value는 탐색할 범위 입니다.

lgbm_parameter_bounds = {

                      'n_estimators' : (30,100),

                      'max_depth' : (1,3), # 나무의 깊이

                      'subsample' : (0.5,1)

                      }



# 함수를 만들어주겠습니다.

# 함수의 구성은 다음과 같습니다.

# 1. 함수에 들어가는 인자 = 위에서 만든 함수의 key값들

# 2. 함수 속 인자를 통해 받아와 새롭게 하이퍼파라미터 딕셔너리 생성

# 3. 그 딕셔너리를 바탕으로 모델 생성

# 4. train_test_split을 통해 데이터 train-valid 나누기

# 5. 모델 학습

# 6. 모델 성능 측정

# 7. 모델의 점수 반환



def lgbm_bo(n_estimators,max_depth, subsample):
  lgbm_params = {
      'n_estimators' : int(round(n_estimators)),
      'max_depth' : int(round(max_depth)),
      'subsample' : int(round(subsample))
  }

  lgbm = LGBMClassifier(**lgbm_params)

  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)

  lgbm.fit(X_train, y_train)

  score = accuracy_score(y_valid, lgbm.predict(X_valid))
  return score


# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요

BO_lgbm = BayesianOptimization(
    f = lgbm_bo, pbounds = lgbm_parameter_bounds, random_state = 0
)



# Bayesian Optimization을 실행해보세요
BO_lgbm.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   | max_depth | n_esti... | subsample |
-------------------------------------------------------------
|  1        |  0.5527   |  2.098    |  80.06    |  0.8014   |
|  2        |  0.5309   |  2.09     |  59.66    |  0.8229   |
|  3        |  0.5345   |  1.875    |  92.42    |  0.9818   |
|  4        |  0.5673   |  1.767    |  85.42    |  0.7644   |
|  5        |  0.5464   |  2.136    |  94.79    |  0.5355   |
|  6        |  0.5682   |  2.913    |  83.67    |  0.7004   |
|  7        |  0.5573   |  1.804    |  30.01    |  0.678    |
|  8        |  0.5409   |  3.0      |  35.75    |  1.0      |
|  9        |  0.5427   |  1.002    |  83.12    |  0.5732   |
|  10       |  0.5745   |  2.986    |  84.93    |  0.9729   |


### Voting Classifier 생성


In [11]:
# 모델 정의 (튜닝된 파라미터로)
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

# VotingClassifier 정의
VC = VotingClassifier(
    estimators = [('rf', RF), ('xgb', XGB), ('lgbm', LGBM)],
    voting = 'soft'
)

## 정답

In [12]:
# X에 학습할 데이터를, y에 목표 변수를 저장해주세요

X = train.drop(columns = ['index', 'quality'])

y = train['quality']



# 랜덤포레스트의 하이퍼 파라미터의 범위를 dictionary 형태로 지정해주세요

## Key는 랜덤포레스트의 hyperparameter이름이고, value는 탐색할 범위 입니다.

rf_parameter_bounds = {

                      'max_depth' : (1,3), # 나무의 깊이

                      'n_estimators' : (30,100),

                      }



# 함수를 만들어주겠습니다.

# 함수의 구성은 다음과 같습니다.
# 1. 함수에 들어가는 인자 = 위에서 만든 함수의 key값들
# 2. 함수 속 인자를 통해 받아와 새롭게 하이퍼파라미터 딕셔너리 생성
# 3. 그 딕셔너리를 바탕으로 모델 생성
# 4. train_test_split을 통해 데이터 train-valid 나누기
# 5 .모델 학습
# 6. 모델 성능 측정
# 7. 모델의 점수 반환



def rf_bo(max_depth, n_estimators):

  rf_params = {
              'max_depth' : int(round(max_depth)),
               'n_estimators' : int(round(n_estimators)),      
              }

  rf = RandomForestClassifier(**rf_params)



  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2, )



  rf.fit(X_train,y_train)

  score = accuracy_score(y_valid, rf.predict(X_valid))

  return score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_rf"라는 변수에 Bayesian Optmization을 저장해보세요

BO_rf = BayesianOptimization(f = rf_bo, pbounds = rf_parameter_bounds,random_state = 0)



# Bayesian Optimization을 실행해보세요

BO_rf.maximize(init_points = 5, n_iter = 5)


|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.5055   |  2.098    |  80.06    |
|  2        |  0.5327   |  2.206    |  68.14    |
|  3        |  0.5345   |  1.847    |  75.21    |
|  4        |  0.5018   |  1.875    |  92.42    |
|  5        |  0.5245   |  2.927    |  56.84    |
|  6        |  0.4709   |  1.018    |  48.01    |
|  7        |  0.4291   |  1.217    |  66.52    |
|  8        |  0.4982   |  1.821    |  75.19    |
|  9        |  0.4891   |  2.199    |  49.04    |
|  10       |  0.5173   |  2.089    |  87.6     |


In [13]:
# X에 학습할 데이터를, y에 목표 변수를 저장해주세요

X = train.drop(columns = ['index', 'quality'])

y = train['quality']



# XGBoost의 하이퍼 파라미터의 범위를 dictionary 형태로 지정해주세요

## Key는 XGBoost hyperparameter이름이고, value는 탐색할 범위 입니다.

xgb_parameter_bounds = {
                      'gamma' : (0,10),
                      'max_depth' : (1,3), 
                      'subsample' : (0.5,1)
                      }



# 함수를 만들어주겠습니다.
# 함수의 구성은 다음과 같습니다.
# 1. 함수에 들어가는 인자 = 위에서 만든 함수의 key값들
# 2. 함수 속 인자를 통해 받아와 새롭게 하이퍼파라미터 딕셔너리 생성
# 3. 그 딕셔너리를 바탕으로 모델 생성
# 4. train_test_split을 통해 데이터 train-valid 나누기
# 5 .모델 학습
# 6. 모델 성능 측정
# 7. 모델의 점수 반환



def xgb_bo(gamma,max_depth, subsample):

  xgb_params = {
              'gamma' : int(round(gamma)),
              'max_depth' : int(round(max_depth)),
               'subsample' : int(round(subsample)),      
              }

  xgb = XGBClassifier(**xgb_params)



  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2, )



  xgb.fit(X_train,y_train)
  score = accuracy_score(y_valid, xgb.predict(X_valid))
  return score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.
# "BO_xgb"라는 변수에 Bayesian Optmization을 저장해보세요

BO_xgb = BayesianOptimization(f = xgb_bo, pbounds = xgb_parameter_bounds,random_state = 0)



# Bayesian Optimization을 실행해보세요

BO_xgb.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   |   gamma   | max_depth | subsample |
-------------------------------------------------------------
|  1        |  0.5291   |  5.488    |  2.43     |  0.8014   |
|  2        |  0.5336   |  5.449    |  1.847    |  0.8229   |
|  3        |  0.55     |  4.376    |  2.784    |  0.9818   |
|  4        |  0.5464   |  3.834    |  2.583    |  0.7644   |
|  5        |  0.5518   |  5.68     |  2.851    |  0.5355   |
|  6        |  0.5455   |  6.692    |  2.421    |  0.9232   |
|  7        |  0.5718   |  2.303    |  2.915    |  0.6916   |
|  8        |  0.5291   |  5.411    |  2.106    |  0.8663   |
|  9        |  0.5636   |  1.798    |  3.0      |  0.7496   |
|  10       |  0.006364 |  2.213    |  2.319    |  0.5      |


In [14]:
# X에 학습할 데이터를, y에 목표 변수를 저장해주세요

X = train.drop(columns = ['index', 'quality'])

y = train['quality']



# LGBM의 하이퍼 파라미터의 범위를 dictionary 형태로 지정해주세요

## Key는 LGBM hyperparameter이름이고, value는 탐색할 범위 입니다.

lgbm_parameter_bounds = {
                      'n_estimators' : (30,100),
                      'max_depth' : (1,3), # 나무의 깊이
                      'subsample' : (0.5,1)
                      }



# 함수를 만들어주겠습니다.

# 함수의 구성은 다음과 같습니다.
# 1. 함수에 들어가는 인자 = 위에서 만든 함수의 key값들
# 2. 함수 속 인자를 통해 받아와 새롭게 하이퍼파라미터 딕셔너리 생성
# 3. 그 딕셔너리를 바탕으로 모델 생성
# 4. train_test_split을 통해 데이터 train-valid 나누기
# 5 .모델 학습
# 6. 모델 성능 측정
# 7. 모델의 점수 반환



def lgbm_bo(n_estimators,max_depth, subsample):

  lgbm_params = {
              'n_estimators' : int(round(n_estimators)),
              'max_depth' : int(round(max_depth)),
               'subsample' : int(round(subsample)),      
              }

  lgbm = LGBMClassifier(**lgbm_params)



  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2, )



  lgbm.fit(X_train,y_train)

  score = accuracy_score(y_valid, lgbm.predict(X_valid))

  return score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요

BO_lgbm = BayesianOptimization(f = lgbm_bo, pbounds = lgbm_parameter_bounds,random_state = 0)



# Bayesian Optimization을 실행해보세요

BO_lgbm.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   | max_depth | n_esti... | subsample |
-------------------------------------------------------------
|  1        |  0.5473   |  2.098    |  80.06    |  0.8014   |
|  2        |  0.5418   |  2.09     |  59.66    |  0.8229   |
|  3        |  0.5427   |  1.875    |  92.42    |  0.9818   |
|  4        |  0.5664   |  1.767    |  85.42    |  0.7644   |
|  5        |  0.5518   |  2.136    |  94.79    |  0.5355   |
|  6        |  0.5273   |  2.338    |  79.72    |  0.9232   |
|  7        |  0.5464   |  1.461    |  97.03    |  0.6916   |
|  8        |  0.5764   |  2.082    |  68.72    |  0.8663   |
|  9        |  0.5391   |  2.029    |  68.71    |  0.8963   |
|  10       |  0.5445   |  1.744    |  38.08    |  0.9286   |


In [15]:
# 모델 정의 (튜닝된 파라미터로)
LGBM = LGBMClassifier(max_depth = 2.09,n_estimators=60, subsample = 0.8229)
XGB = XGBClassifier(gamma =  4.376, max_depth = 2.784, subsample = 0.9818)
RF = RandomForestClassifier(max_depth = 3.0, n_estimators = 35.31)

# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

# [↩️ 리스트로 돌아가기](https://dacon.io/competitions/open/235698/overview/description)